In [1]:
!pip install apache-beam
!pip install great_expectations
!pip install apache-airflow
!pip install apache-flink
!pip install great_expectations==0.15.49
!great_expectations init
!pip install pandas
!pip install apache-beam
!pip install great_expectations
!pip install apache-airflow
!pip install apache-flink
!pip install great_expectations==0.15.49
!great_expectations init

  Using cached ruamel.yaml-0.17.17-py3-none-any.whl.metadata (12 kB)
Using cached ruamel.yaml-0.17.17-py3-none-any.whl (109 kB)
  Attempting uninstall: ruamel.yaml
    Found existing installation: ruamel.yaml 0.18.10
    Uninstalling ruamel.yaml-0.18.10:
      Successfully uninstalled ruamel.yaml-0.18.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-flink 1.20.0 requires ruamel.yaml>=0.18.4, but you have ruamel-yaml 0.17.17 which is incompatible.
  Using cached protobuf-5.29.3-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
Using cached protobuf-5.29.3-cp38-abi3-manylinux2014_x86_64.whl (319 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.23.4
    Uninstalling protobuf-4.23.4:
      Successfully uninstalled protobuf-4.23.4
ERROR: pip's dependency resolver does not currently take into account all the packages tha

In [2]:
import requests
import json
import apache_beam as beam
import psycopg2
import great_expectations as ge
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime, timedelta
import logging

# --- Configurations ---
APP_ID = "YOUR_APP_ID"
API_KEY = "YOUR_API_KEY"
PRIMARY_DB_CONN_STR = "YOUR_AZURE_POSTGRESQL_CONN_STR"
SECONDARY_DB_CONN_STR = "YOUR_LOCAL_POSTGRESQL_CONN_STR"
GE_CONTEXT_ROOT_DIR = "./great_expectations"
STATION_IDS = ["STATION_ID1", "STATION_ID2", "STATION_ID3"]
EXPECTATION_SUITE_NAME = "train_timetable_expectations"

# --- Logging Setup ---
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

**2. Data Ingestion
2.1. Real-Time Data (Apache Flink)**

In [3]:
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.table import StreamTableEnvironment

# Initialize Flink environment
env = StreamExecutionEnvironment.get_execution_environment()
t_env = StreamTableEnvironment.create(env)

# Define a source table for real-time data
t_env.execute_sql("""
    CREATE TABLE train_departures (
        station STRING,
        train_uid STRING,
        aimed_departure_time TIMESTAMP,
        status STRING
    ) WITH (
        'connector' = 'kafka',
        'topic' = 'train_departures',
        'properties.bootstrap.servers' = 'localhost:9092',
        'format' = 'json'
    )
""")

**. Scheduled Data (Python requests)**

In [3]:
def get_train_data(station_id):
    url = f"https://transportapi.com/v3/uk/train/station/{station_id}/live.json?app_id={APP_ID}&app_key={API_KEY}&from_offset=0&to_offset=30&live=true"
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        logger.error(f"API request failed for {station_id}: {e}")
        return None

**3. Data Validation (Great Expectations)**

In [4]:
# Initialize Great Expectations context
context = ge.DataContext(GE_CONTEXT_ROOT_DIR)

def validate_data_with_ge(data):
    if not data:
        return None
    batch = context.get_batch(
        batch_kwargs={"dataset": data},
        expectation_suite_name=EXPECTATION_SUITE_NAME
    )
    results = context.run_validation_operator("action_list_operator", assets_to_validate=[batch])
    if not results.success:
        logger.warning("Data validation failed!")
        return None
    return data

[2025-02-07T21:28:15.694+0000] {file_data_context.py:169} INFO - FileDataContext loading zep config
[2025-02-07T21:28:15.705+0000] {config.py:101} INFO - GxConfig.parse_yaml() failed with errors - [{'loc': ('xdatasources',), 'msg': 'field required', 'type': 'value_error.missing'}]
[2025-02-07T21:28:15.707+0000] {config.py:105} INFO - GxConfig.parse_yaml() returning empty `xdatasources`
[2025-02-07T21:28:15.709+0000] {config.py:53} INFO - Loading 'datasources' ->
{}
[2025-02-07T21:28:15.712+0000] {config.py:79} INFO - Loaded 'datasources' ->
{}


**4. Data Transformation (Apache Beam)**

In [5]:
def transform_data(data):
    if not data:
        return None
    transformed = []
    for train in data.get("departures", {}).get("all", []):
        transformed.append({
            "station": data.get("station_name"),
            "train_uid": train.get("train_uid"),
            "aimed_departure_time": train.get("aimed_departure_time"),
            "expected_departure_time": train.get("expected_departure_time"),
            "status": train.get("status"),
        })
    return transformed

**5. Data Storage (PostgreSQL)**

In [6]:
def load_to_postgres(data, conn_str):
    if not data:
        return
    try:
        conn = psycopg2.connect(conn_str)
        cursor = conn.cursor()
        for train in data.get("departures", {}).get("all", []):
            cursor.execute(
                """
                INSERT INTO train_timetable (station, train_uid, aimed_departure_time, status)
                VALUES (%s, %s, %s, %s)
                ON CONFLICT (train_uid) DO UPDATE SET status = EXCLUDED.status
                """,
                (data.get("station_name"), train.get("train_uid"), train.get("aimed_departure_time"), train.get("status"))
            )
        conn.commit()
        cursor.close()
        conn.close()
    except Exception as e:
        logger.error(f"Error inserting data into PostgreSQL: {e}")

**6. Apache Beam Pipeline**

In [7]:
def run_beam_pipeline():
    with beam.Pipeline() as pipeline:
        (
            pipeline
            | "Create Station IDs" >> beam.Create(STATION_IDS)
            | "Fetch Train Data" >> beam.Map(get_train_data)
            | "Validate Data" >> beam.Map(validate_data_with_ge)
            | "Transform Data" >> beam.Map(transform_data)
            | "Load to Primary DB" >> beam.Map(lambda data: load_to_postgres(data, PRIMARY_DB_CONN_STR))
            | "Load to Backup DB" >> beam.Map(lambda data: load_to_postgres(data, SECONDARY_DB_CONN_STR))
        )

**7. Airflow DAG Orchestration**

In [8]:
# Airflow DAG configuration
default_args = {
    "owner": "airflow",
    "depends_on_past": False,
    "start_date": datetime(2024, 2, 1),
    "retries": 2,
    "retry_delay": timedelta(minutes=10),
}

# Define the DAG
with DAG(
    dag_id="train_data_pipeline",
    schedule_interval="@hourly",
    default_args=default_args,
    catchup=False,
) as dag:
    ingest_data_task = PythonOperator(
        task_id="ingest_data",
        python_callable=run_beam_pipeline,
    )

<ipython-input-8-0ca753a15ce3>:11 RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead.


**8. Monitoring & Alerting**

In [9]:
from airflow.utils.email import send_email

def notify_failure(context):
    subject = f"Airflow Task Failed: {context.get('task_instance').task_id}"
    body = f"Task failed with exception: {context.get('exception')}"
    send_email(to="your-email@example.com", subject=subject, html_content=body)

# Add failure callback to default_args
default_args["on_failure_callback"] = notify_failure

**9. Main Execution**

In [3]:
import apache_beam as beam

def run_beam_pipeline():
    with beam.Pipeline() as pipeline:
        (
            pipeline
            | "Create Station IDs" >> beam.Create(STATION_IDS)
            | "Fetch Train Data" >> beam.Map(get_train_data)
            | "Validate Data" >> beam.Map(validate_data_with_ge)
            | "Transform Data" >> beam.Map(transform_data)
            | "Load to Primary DB" >> beam.Map(lambda data: load_to_postgres(data, PRIMARY_DB_CONN_STR))
            | "Load to Backup DB" >> beam.Map(lambda data: load_to_postgres(data, SECONDARY_DB_CONN_STR))
        )